In [ ]:
#!/bin/bash
import os
import shutil
import re
from pathlib import Path

# A module for surgical replacement of elements within files of a directory tree, 
# and creating a forked copy in the process. 

# Settings (adjust these paths accordingly)
hostfs_vault_path="/path/to/your/vault" 
cpfs_path="/path/to/cpfs"

# Explanation of cfs (copy file-system): Shutil copy of original host file system with all transfomrations enabled. cfs will do `git add .`, `git commit -m "message"`, and `git push` to the remote repository prior to the end of the script to prepare for the (not-listed) hfs_method() to read the git log and extract the handful of changes that WILL be made to the hfs.

# Explanation of hfs (copy file-system): Host file system (hfs) is the original file system and will be untouched. This module is about surgical replacment of a few lines on this hfs but those operations are not in main.py and must be invoked manually with no recursion.

# Explanation of hfs_method(): This method will read the git log of the cfs and extract the handful of changes that WILL be made to the hfs. This method will then apply those changes to the hfs once a git commit is made to the cfs (is confirmed to be successful, and valid hfs changes need to bemade).

# Forked Copy (cpfs): The creation of a dynamically generated cpfs that serves as a mutable workspace derived from the original file system.


# * cpfs (copy file-system): A mutable copy generated at run-time, serving as the 
#    basis for modifications.
# * hfs (host file-system): The original file system.  
#     * File Structure: Immutable.
#     * Metadata: Immutable (except for unavoidable OS-level modifications).
#     * File Contents: Minimally mutable. Acceptable changes must be confined to single-line 
#        statements representing UTF-8 path strings (as used by Obsidian for links).  
# * Target Elements:  Obsidian-generated path/link strings within files.

# a module for surgical replacment elements of files within a directory tree, and creating a forked copy in the process. 
def migrate_media(vault_path, media_dir=".media"):
    """
    Migrates linked media files in an Obsidian vault to a dedicated media directory.

    Args:
        vault_path (str): Path to your Obsidian vault.
        media_dir (str): Name of the media directory (relative to the vault).
    """

    media_path = Path(vault_path) / media_dir
    media_path.mkdir(exist_ok=True)  # Create the media directory if needed

    # Iterate over Markdown files in the vault
    for md_file in vault_path.rglob("*.md"):
        with md_file.open("r+") as f:  # Open in read/write mode
            content = f.read()

            # Find linked media files (adjust the pattern if needed)
            for match in re.finditer(r"!\[\[(.*?)\]\]", content):
                original_path = Path(vault_path) / match.group(1)
                if original_path.exists():
                    # Calculate new filename (add hashing, metadata if desired)
                    new_filename = original_path.name 
                    new_path = media_path / new_filename

                    # Move the file
                    shutil.move(original_path, new_path)

                    # Update the link in the Markdown file
                    new_link = f"![[{media_dir}/{new_filename}]]"
                    content = content.replace(match.group(0), new_link)

            f.seek(0)  # Reset file pointer
            f.write(content)
            f.truncate() 

if __name__ == "__main__":
    vault_path = input("Enter your Obsidian vault path: ")
    migrate_media(vault_path)
